<a href="https://colab.research.google.com/github/MRPatrick8/NLP/blob/main/Hackathon/Hackathon_File.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Installation zone**

In [ ]:
!pip install fastapi pyngrok uvicorn nest-asyncio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 4.5 MB/s 
     |████████████████████████████████| 745 kB 42.7 MB/s 
     |████████████████████████████████| 56 kB 5.9 MB/s 
     |████████████████████████████████| 64 kB 3.4 MB/s 
     |████████████████████████████████| 80 kB 11.5 MB/s 
     |████████████████████████████████| 58 kB 7.7 MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.1.0-py3-none-any.whl size=19007 sha256=07973b6bc924ec72ef718b2fe8bd4aa6e67b17450f9e04fd27bbd402c86e05b8
  Stored in directory: /root/.cache/pip/wheels/bf/e6/af/ccf6598ecefecd44104069371795cb9b3afbcd16987f6ccfb3
Successfully built pyngrok


In [ ]:
!pip install -U easynmt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 32.2 MB/s 
     |████████████████████████████████| 1.3 MB 49.4 MB/s 
     |████████████████████████████████| 68 kB 7.8 MB/s 
     |████████████████████████████████| 163 kB 47.7 MB/s 
     |████████████████████████████████| 7.6 MB 24.0 MB/s 
  Using cached pybind11-2.10.1-py3-none-any.whl (216 kB)
  Created wheel for easynmt: filename=EasyNMT-2.0.2-py3-none-any.whl size=19919 sha256=25c6ae2fe9821760fa35a14e2469983c756e657cd0929da2b6a3f8cb0a437170
  Stored in directory: /root/.cache/pip/wheels/d1/57/06/53ca38645e14d4537a41e5a36da2026e10f54ae88240dd5190
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3157963 sha256=ea079b45565a20eb025debfca53ba386737f2704272526350e338d93aef3ee9d
  Stored in directory: /root/.cache/pip/wheels/4e/ca/bf/b020d2be95f7641801a6597a29c8f4f19e38f9c02a345bab9b
Successfully built easy

**All required imports**

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from fastapi import FastAPI, Response
from pyngrok import ngrok
import nest_asyncio
import uvicorn
from starlette.responses import HTMLResponse
import json
from easynmt import EasyNMT

**other prequisties**

In [ ]:
!ngrok config add-authtoken 2HJQ7EKoI9F4HIlHhcus0P1VEnK_7svYR7todWWUT5MHmM1Sb

NAME:
   ngrok - tunnel local ports to public URLs and inspect traffic

DESCRIPTION:
    ngrok exposes local networked services behinds NATs and firewalls to the
    public internet over a secure tunnel. Share local websites, build/test
    webhook consumers and self-host personal services.
    Detailed help for each command is available with 'ngrok help <command>'.
    Open http://localhost:4040 for ngrok's web interface to inspect traffic.

EXAMPLES:
    ngrok http 80                    # secure public URL for port 80 web server
    ngrok http -subdomain=baz 8080   # port 8080 available at baz.ngrok.io
    ngrok http foo.dev:80            # tunnel to host:port instead of localhost
    ngrok http https://localhost     # expose a local https server
    ngrok tcp 22                     # tunnel arbitrary TCP traffic to port 22
    ngrok tls -hostname=foo.com 443  # TLS traffic for foo.com to port 443
    ngrok start foo bar baz          # start tunnels from the configuration file

VERSI

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#PART 1: WEB SCRAPING OF JOB IN RWANDA

In [ ]:
# SOUP
url = 'https://www.jobinrwanda.com/jobs/featured'
soup = BeautifulSoup(requests.get(url).content, 'html.parser')

In [ ]:
# LINKS OF ALL FEATURED CONTENT ON WEB
job_titles = {}

parent_divs = soup.find_all('div', class_ = 'card-body p-2')
featured_links = []

for index,div in enumerate(parent_divs):
  anchor = div.find('a')
  output_link = 'https://www.jobinrwanda.com'+anchor['href']
  job_titles[index] = anchor.find('h5', class_ = 'card-title').get_text()
  featured_links.append(output_link)
job_titles = list(job_titles.values())

In [ ]:
# EACH LINK SOUP
soups2 = {}
for index,link in enumerate(featured_links):
  SOUP = BeautifulSoup(requests.get(link).content, 'html.parser')
  soups2[index] = SOUP

In [ ]:
# DIVS FOR EACH FEATURE CONTENT
description_divs = {}
job_summary = {}
for index,level2_soup in enumerate(list(soups2.values())):
  description_divs[index] = level2_soup.find('div', class_ = 'clearfix text-formatted field field--name-field-job-full-description field--type-text-long field--label-hidden field__item')
  ul = level2_soup.find('ul', class_ = 'list-group list-group-flush')
  info = {}
  for li in ul.children:
    try:
      summary = re.sub('\s{2,}|\n',' ',li.text) # both key and value together as str
      # separate key & value to get key:value dictionary
      if not 'apply' in summary.lower():
        if 'viewed' in summary:
          times = re.search('\d+', summary).group()
          info['views'] = times
        else:
          k,value = re.split(':', summary)[:2]
          info[k.lstrip()] = value
    except AttributeError:
      pass
  # info["application link"] = app_link
  # # print(info)
  job_summary[index] = info


In [ ]:
# FULL CONTENT OF EACH FEATURE
full_description = {}
for index,div in enumerate(list(description_divs.values())):
  single_div_content = []
  for para in div.find_all('p', class_ = 'text-align-justify'):
    single_div_content.append(para.get_text())
  full_description[index] = re.sub('\n|\\xa0','','\n'.join(single_div_content))

In [ ]:
# CREATE THE JOB ROW TO BE USED IN THE DATAFRAME
job_info_rows = []
for index in list(full_description.keys()):
  summary_info = list(job_summary[index].values())
  descr_text = [full_description[index]]
  job_info_rows.append(summary_info + descr_text)


In [ ]:
# CREATE DATAFRAME
columns = list(job_summary[0].keys()) + ['Description']
df1 = pd.DataFrame(job_info_rows, columns = columns)
df1['job title'] = job_titles
df1['job title'] = df1['job title'].str.replace('\n','',regex=True).astype('str')

In [ ]:
df1.head()

,views,Location,Sector,Education level,Desired experience,Contract type,Deadline,Number of positions,Description,job title
0,15,", RW","Marketing and sales, Other",Bachelor,Mid career (3 to 5 years of experience),Full-time,"Monday, 21/11/2022 23",1,Graphic DesignerEmployment type –FULL TIMEAbou...,Graphic Designer
1,62,"KIGALI , RW",Consultancy,Not specified,No experience,Other,"Friday, 16/12/2022 18",50,Call for Applications – Business Trainings for...,Call for Applications: Business Trainings for...
2,365,", RW","Accounting, Finance and investment, Other",Bachelor,Mid career (3 to 5 years of experience),Full-time,"Sunday, 20/11/2022 23",1,"On behalf of our client, We are looking for an...",Internal Audit Manager
3,182,", RW",Other,Bachelor,Senior (5+ years of experience),Contract,"Monday, 21/11/2022 09",1,Re-advertisement - Production of Radio Drama S...,Production of Radio Drama Series for Behavior ...
4,403,", RW",Other,Not specified,Not specified,Contract,"Friday, 25/11/2022 10",1,RE-ADVERTISEMENT OFTENDER : Nº: 022/10/2022/BR...,Hiring a Firm for BRD Maintenance Assessment


In [ ]:
# FILTER FOR ONLY IT RELATED JOBS & CONSULTANCY
IT_based_jobs = df1.copy()
IT_based_jobs['is IT job'] = False

for index in range(len(IT_based_jobs)):
  if re.findall('Computer|IT|Software', IT_based_jobs.iloc[index]['Sector']):
    IT_based_jobs['is IT job'][index] = True

IT_based_jobs = IT_based_jobs[IT_based_jobs['is IT job'] == True]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [ ]:
# display IT jobs 
IT_based_jobs.head()

,views,Location,Sector,Education level,Desired experience,Contract type,Deadline,Number of positions,Description,job title,is IT job
48,440,"Kigali , RW","Computer and IT, Management, Project management",Bachelor,Senior (5+ years of experience),Full-time,"Tuesday, 10/01/2023 23",1,Do you want to do work that matters? Do you wa...,Product Manager,True
130,1607,"Kigali , RW","Computer and IT, Other, Project management",Bachelor,Senior (5+ years of experience),Full-time,"Friday, 18/11/2022 23",1,CHANCEN International is a non-profit organiza...,IT and MIS Director,True
135,374,"Kigali , RW","Agriculture, Agronomy, Business, Computer and...",Bachelor,Mid career (3 to 5 years of experience),Full-time,"Wednesday, 25/01/2023 23",1,"About One Acre FundFounded in 2006, One Acre F...",Global Remote Sensing Manager,True
140,882,", RW","Computer and IT, Education, Other",Bachelor,Not specified,Full-time,"Monday, 28/11/2022 23",1,DescriptionJob Title:Instructional Technology ...,Instructional Technology Assistant,True
167,2844,"Kigali , RW","Agriculture, Business, Computer and IT, Manag...",Not specified,Not specified,Full-time,"Tuesday, 06/12/2022 23",1,"About One Acre FundFounded in 2006, One Acre F...",IT Operations Senior Manager,True


#PART 2 : WEBSCRAP umucyo.gov.rw

In [ ]:
url = 'https://www.umucyo.gov.rw/eb/bav/selectListAdvertisingListForGU.do'

text = """"""
with open('/content/drive/MyDrive/NLP/Data/Umucyo.txt', 'r') as f:
  data = f.readlines()
  for line in data:
    text += line

# umucyo_soup = BeautifulSoup(requests.get(url).content, 'html.parser')
umucyo_soup = BeautifulSoup(text, 'html.parser')
table = umucyo_soup.find('table', class_ = 'article_table mb10')

# CREATE COLUMNS FOR THE DATAFRAME
head_row = table.find('thead').find('tr')
titles = []
for th in head_row.find_all('th'):
  titles.append(re.sub('\r|\n|\t','',th.text))
titles = titles[1:]

In [ ]:
# MAKE DATAFRAME BODY
body_rows = table.find('tbody').find_all('tr') 
full_data = []
for row in body_rows:
  ls = []
  for detail in row.find_all('td'):
    ls.append(re.sub('\n|\r|\t|\s', '',detail.text))
  full_data.append(ls[1:])


In [ ]:
# CREATE UMUCYO DATAFRAME
umucyo_df = pd.DataFrame(full_data, columns = titles)

In [ ]:
umucyo_df

,Tender Name,Tender No,Status,Advertising Date,Deadline of Submitting,Planed Open Date,Stage Type
0,HiringconsultantUrbanMobility(Publictransport)...,000008/C/ICB/2022/2023/CoK,Published,10/11/2022,27/12/202210:00,27/12/202210:30,onestage
1,HiringconsultantTransporteconomisttosupportRwa...,000009/C/ICB/2022/2023/CoK,Published,10/11/2022,27/12/202210:00,27/12/202210:30,onestage
2,sciencekitsandlaboratoryequipment,000001/G/ICB/2022/2023/REB,Published,03/11/2022,20/12/202210:00,20/12/202210:30,onestage
3,HiringaninternationalExpertinDataProcessingtos...,000008/C/ICB/2022/2023/NISR,Published,02/11/2022,19/12/202210:00,19/12/202210:15,onestage
4,TENDERFOROFFICESUPPLIESANDSTATIONARY,000001/G/NCB/2022/2023/Kiziguro,Published,04/11/2022,16/12/202210:00,16/12/202210:30,onestage
5,supplyoflabokitsreagentsandconsumables,000034/G/NCB/2022/2023/RAB,Published,10/11/2022,15/12/202211:00,15/12/202211:30,onestage
6,Provisionofservicesofraisingawarenessonstandar...,000010/NC/NCB/2022/2023/RSB,Published,11/11/2022,15/12/202210:00,15/12/202210:10,onestage
7,"Rehabilitationofadministrationbloc,constructio...",000001/W/NCB/2022/2023/NIDA,Published,09/11/2022,15/12/202210:00,15/12/202210:30,onestage
8,HiringaconsultantfirmforStudyreviewandsupervis...,000014/C/NCB/2022/2023/CoK,Published,11/11/2022,15/12/202209:00,15/12/202209:10,onestage
9,Supplyofcalibrationcertifiedreferencematerials...,000009/G/NCB/2022/2023/RSB,Published,11/11/2022,15/12/202209:00,15/12/202209:30,onestage


#PART 3 : DISPLAY DATA ON THE BROWSER

In [ ]:
model = EasyNMT('opus-mt')
app = FastAPI(title = "to get something you never had you have do something you never did")

11.9kB [00:00, 2.71MB/s]                   


**PART 3.1. DISPLAY JSON DATA OF DATAFRAMES**

In [ ]:
@app.get('/')
def homepage():
  content = f"""<html>
                    <h1>Welcome to Rene Patrick API for the first hackathon</h1>
                    <p>
                      Manners Make a Man
                    </p>
                    <p>
                      <strong>never give up! \U0001F680 \U0001F680 \U0001F680 </strong>
                    </p>
                </html>
              """ 
  return HTMLResponse(content)

In [ ]:
# DISPLAY ALL IT JOBS ON THE BROWSER
@app.get('/IT')
def show(orient:str='index'):
    return Response(IT_based_jobs.to_json(orient = orient), media_type = 'application/json')

In [ ]:
# SHOW TO THE BROWSER ALL INFORMATIONS RETRIEVED FROM JOBINRWANDA
@app.get('/jobs')
def showall(orient:str='index'):
    return Response(df1.to_json(orient = orient), media_type = 'application/json')

In [ ]:
# SHOW ALL UMUCYO INFORMATION TO THE BROWSER
@app.get('/umucyo_tenders')
def show_umucyo(orient:str='index'):
    return Response(umucyo_df.to_json(orient = orient), media_type = 'application/json')

**PART 3.2. TRANSLATE ALL IT RELATED JOBS DATAFRAME COLUMNS DESCRIPTION AND JOB TITLE**

In [ ]:
# TRANSLATE ALL IT RELATED JOBS IN DIFFERENT LANGUANGES
prev_lang = 'en'
@app.get('/translate')
def dr(lang:str=''):
  global prev_lang
  success = True
  if lang:
    try:
      d = []
      f = []
      for index in range(len(IT_based_jobs)):
        text1 = IT_based_jobs.iloc[index]['Description']
        text2 = IT_based_jobs.iloc[index]['job title']
        d.append(model.translate(text1, source_lang = prev_lang, target_lang = lang))
        f.append(model.translate(text2, source_lang = prev_lang, target_lang = lang))

        if index != len(IT_based_jobs) - 1:
          print(f"{index} / {len(IT_based_jobs)}")
           
      IT_based_jobs['job title'] = f
      IT_based_jobs['Description'] = d
      success = True
      prev_lang = lang
    except:
      error_message = f"""<html>
                            <h2>
                              Oops! Something went wrong. <br> Try to choose other language maybe!.
                            </h2>
                          </html>"""
      success = False
      return HTMLResponse(error_message)
  if success:
    return Response(IT_based_jobs.to_json(orient = 'index'), media_type = 'application/json')

In [ ]:
# RUN THE SERVER
ngrok_tunnel = ngrok.connect(8000)
print("Your public API URL:", ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port = 8000)

Your public API URL: http://02bb-35-245-186-127.ngrok.io


INFO:     Started server process [76]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     102.22.160.160:0 - "GET / HTTP/1.1" 200 OK
INFO:     102.22.160.160:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     102.22.160.160:0 - "GET / HTTP/1.1" 200 OK
INFO:     102.22.160.160:0 - "GET /jobs HTTP/1.1" 200 OK
INFO:     102.22.160.160:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     102.22.160.160:0 - "GET /translate HTTP/1.1" 200 OK


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Downloading:   0%|          | 0.00/778k [00:00<?, ?B/s]

...

/usr/local/lib/python3.7/dist-packages/transformers/models/marian/tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


Downloading:   0%|          | 0.00/301M [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1364: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 512 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


0 / 5
1 / 5
2 / 5
3 / 5
INFO:     102.22.160.160:0 - "GET /translate?lang=fr HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [76]


In [ ]:
cs = IT_based_jobs.to_csv( "mycsv.csv")